In [57]:
import pandas as pd
import numpy as np
# %matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

In [58]:
# # Max price in 7 days - 1st opening price
# Y is divided into 2 groups, (negative & zero) or (positive value) 

# Read the stock price and the reward rate

# 2016/06/01-2017/10/30
# TWS100

In [59]:
TW100 = pd.read_excel("TW100NMB.xlsx")#Taiwan100 Stock Price Data

In [60]:
TW100Y = pd.read_excel("TW100Y.xlsx")

In [61]:
X_Test = pd.read_excel("Test_X.xlsx")

In [62]:
Y_Test = pd.read_excel("Test_Y.xlsx")

In [63]:
#TW100.head()

In [64]:
#X_Test.head()

In [65]:
TW100NT = TW100.drop(["年月日","市值(百萬元)"] ,axis=1)

In [66]:
X_TestSP = X_Test.drop(["年月日"] ,axis=1)

In [67]:
X_Train_SP = TW100NT.values

In [68]:
X_Test_SP = X_TestSP.values 

In [69]:
Y_Train_MAX_1OPpct = TW100Y["預測最大漲幅"]
Y_Train_MIN_1OPpct = TW100Y["預測最大跌幅"]

In [70]:
Y_1_Train_MAX_1OPpct = Y_Train_MAX_1OPpct.values
Y_1_Train_MIN_1OPpct = Y_Train_MIN_1OPpct.values

In [71]:
Y_Test_MAX_1OPpct = Y_Test["預測一周漲幅"]
Y_Test_MIN_1OPpct = Y_Test["預測一周跌幅"]

In [72]:
Y_1_Test_MAX_1OPpct = Y_Test_MAX_1OPpct.values
Y_1_Test_MIN_1OPpct = Y_Test_MIN_1OPpct.values

In [73]:
L = [];

In [74]:
for j in range(0,34500,345):
    for i in range(309):
        a = X_Train_SP[(i+j):(i+j+30)];
        L.append(a);
    L

In [75]:
L1 = np.reshape(L,(30900,240,1))

In [76]:
M = []

In [77]:
for j in range(0,17250,345):
    for i in range(309):
        a = X_Test_SP[(i+j):(i+j+30)];
        M.append(a);
    M

In [78]:
M1 = np.reshape(M,(15450,240,1))

In [79]:
from scipy import stats

In [80]:
stats.describe(Y_1_Train_MIN_1OPpct)

DescribeResult(nobs=30900, minmax=(-21.465201465201464, 0.0), mean=-2.2232048877240485, variance=4.8417881703502808, skewness=-2.1733641040208727, kurtosis=6.89681776091855)

In [81]:
#Catogorize y
Y_1_Train_MIN_1OPpct[(Y_1_Train_MIN_1OPpct>-0.5) & (Y_1_Train_MIN_1OPpct<=0)] = 0
Y_1_Train_MIN_1OPpct[(Y_1_Train_MIN_1OPpct>-1.5) & (Y_1_Train_MIN_1OPpct<=-0.5)] = 1
Y_1_Train_MIN_1OPpct[(Y_1_Train_MIN_1OPpct>-5) & (Y_1_Train_MIN_1OPpct<=-1.5)] = 2
Y_1_Train_MIN_1OPpct[(Y_1_Train_MIN_1OPpct>-10) & (Y_1_Train_MIN_1OPpct<=-5)] = 3
Y_1_Train_MIN_1OPpct[(Y_1_Train_MIN_1OPpct<=-10)] = 4

In [82]:
#Catogorize y
Y_1_Test_MIN_1OPpct[(Y_1_Test_MIN_1OPpct>-0.5) & (Y_1_Test_MIN_1OPpct<=0)] = 0
Y_1_Test_MIN_1OPpct[(Y_1_Test_MIN_1OPpct>-1.5) & (Y_1_Test_MIN_1OPpct<=-0.5)] = 1
Y_1_Test_MIN_1OPpct[(Y_1_Test_MIN_1OPpct>-5) & (Y_1_Test_MIN_1OPpct<=-1.5)] = 2
Y_1_Test_MIN_1OPpct[(Y_1_Test_MIN_1OPpct>-10) & (Y_1_Test_MIN_1OPpct<=-5)] = 3
Y_1_Test_MIN_1OPpct[(Y_1_Test_MIN_1OPpct<=-10)] = 4

In [83]:
# %matplotlib inline
# #開始觀察資料決定區間
# bins = np.arange(-22, 0, 1)
# plt.hist(Y_1_Train_MIN_1OPpct, bins = bins, alpha = 0.5)

In [84]:
#將Ｙ轉換型態
from keras.utils import np_utils
y_train_trans = np_utils.to_categorical(Y_1_Train_MIN_1OPpct,5)
y_test_trans = np_utils.to_categorical(Y_1_Test_MIN_1OPpct,5)

In [85]:
### Build the model

In [86]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding, Flatten
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

In [87]:
model = Sequential()
model.add(Conv1D(64, 8, activation='relu', input_shape=(L1.shape[1],L1.shape[2])))
model.add(Conv1D(64, 8, activation='relu'))
model.add(MaxPooling1D(8))
model.add(Conv1D(128, 8, activation='relu'))
model.add(Conv1D(128, 8, activation='relu'))
model.add(GlobalAveragePooling1D())
#model.add(Dropout(0.2))
#model.add(Flatten())
model.add(Dense(y_train_trans.shape[1], activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 233, 64)           576       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 226, 64)           32832     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 28, 64)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 21, 128)           65664     
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 14, 128)           131200    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645       
Total para

In [88]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ["accuracy"])

In [89]:
#from keras.callbacks import ModelCheckpoint

In [90]:
#checkpointer = ModelCheckpoint(filepath="/home/llw/Downloads/weights.hdf5", verbose=1, save_best_only=True)

In [91]:
model.fit(L1, y_train_trans, batch_size=16, epochs=1, validation_data=(M1, y_test_trans))# callbacks=[checkpointer]

Train on 30900 samples, validate on 15450 samples
Epoch 1/1
30900/30900 [==============================] - 79s - loss: nan - acc: 0.1765 - val_loss: nan - val_acc: 0.1804


In [93]:
score = model.evaluate(M1, y_test_trans)
score

15424/15450 [============================>.] - ETA: 0s

[nan, 0.18038834953771055]

In [94]:
pick = np.random.randint(1,15450, 20)

In [95]:
result = model.predict_classes(M1)

15392/15450 [============================>.] - ETA: 0s

In [96]:
from ipywidgets import interact_manual

In [97]:
def identify(testnumber):
    print(testnumber);
    print("神經網路判斷為：",result[testnumber]);
    print("正確答案是:",y_test_trans[testnumber]);

In [98]:
interact_manual(identify, testnumber=(0,15449))

10089
神經網路判斷為： 0
正確答案是: [ 0.  0.  0.  1.  0.]


In [99]:
list = []
for i in range(15449):
    #if (result[i] == 0) and (Y_1_Test_7DRR[i] == 0):
        #list.append(i);
    if (result[i] == 1) and (Y_1_Test_MIN_1OPpct[i] == 1):
        list.append(i);

In [100]:
len(list)

0